<a href="https://colab.research.google.com/github/MOOwuttichai/Cancer_Social_Listening/blob/main/%E0%B8%9A%E0%B8%AD%E0%B8%81%E0%B9%82%E0%B8%A3%E0%B8%84%2B%E0%B9%80%E0%B8%9E%E0%B8%A8%2B%E0%B9%83%E0%B8%84%E0%B8%A3%E0%B9%80%E0%B8%9B%E0%B9%87%E0%B8%99%E0%B8%84%E0%B8%99%E0%B9%80%E0%B8%A5%E0%B9%88%E0%B8%B2%2BSentenceTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install attacut
!pip install pythainlp
!pip install epitran
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56

In [ ]:
import pandas as pd
from google.colab import drive
from attacut import tokenize, Tokenizer
import pythainlp
from pythainlp import sent_tokenize, word_tokenize, Tokenizer
from pythainlp.util import normalize
from pythainlp.corpus.common import thai_words
from collections import OrderedDict
import matplotlib as mpl
import numpy as np
from sklearn.cluster import KMeans
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Carbon/data/data_commentsFB_docter.csv')

In [ ]:
data.rename(columns={'name': 'ชื่อ', 'comments': 'คำพูดโรค'}, inplace=True)

In [ ]:
data

,ชื่อ,คำพูดโรค
0,ศ.นพ.ชวลิต ศูนย์มะเร็ง รพ.จุฬาลงกรณ์,กดที่ลิงค์นี้เพื่อเข้าไปอ่านคอมเม้นของโพสต์ครั...
1,Nubeer Kithiya,เป็นมะเร็งลำไส้ใหญ่ ระยะที่ 3 ตอนอายุ 26 ค่ะ พ...
2,Pam Prechanont,คุณแม่เป็นมะเร็งเม็ดเลือดขาวแบบเฉียบพลันค่ะ คุ...
3,Ize Atitya,ลูกสาว 5 ขวบ เป็นมะเร็งสมองคะ ไม่มีอาการอะไร น...
4,นู๋นิด ขนิษฐา,ขอให้เรื่องราวของพ่อกับพี่ชายเราเป็นวิทยาทานแก...
...,...,...
1661,Ba'ra Reer,Chanakan Phetpisal มาอ่าน
1662,Jirawat Tee,Jitladda Thonglukkeaw มาแชเร้ว ได้บุนนะ
1663,Punnapa Kampinitkulladet,Kamta Auttakan
1664,Chanikarn Tinmuean,อนุชา มีลาภ


In [ ]:
comment=data.groupby('ชื่อ').sum().reset_index()

In [ ]:
words = set(thai_words())
words.remove("โรคมะเร็ง")
name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร','ต่อมไทรอยด์','ต่อมไทยรอยด์','ท่อน้ำดี']
for i in name:
  words.add(i)
custom_tokenizer = Tokenizer(words)

In [ ]:
comment['คำพูดโรค'][0]

'ไม่ทานเนื้อค่ะ เห็นญาติพี่น้องรอบข้างเป็นมะ เยอะค่ะ'

In [ ]:
name_cancar = ['มะเร็งกระเพาะปัสสวะ','มะเร็งเยื่อบุโพรงมดลูก','มะเร็งปากมดลูก','มะเร็งเม็ดเลือดขาว','มะเร็งตับ','มะเร็งตับอ่อน','มะเร็งต่อมลูกหมาก','มะเร็งกระเพาะปัสสาวะ','มะเร็งกระเพาะอาหาร'
,'มะเร็งหลอดอาหาร','มะเร็งเต้านม','มะเร็งปอด','มะเร็งลำไส้ใหญ่','มะเร็งรังไข่','มะเร็งช่องปาก','มะเร็งต่อมไทรอยด์','มะเร็งต่อมน้ำเหลือง','มะเร็งสมอง','มะเร็งผิวหนัง','มะเร็งลำไส้'
,'มะเร็งท่อน้ำดี',"มะเร็งลำไส้ใหญ่และทวารหนัก","มะเร็งผิวหนังชนิดเมลาโนมา","มะเร็งต่อมน้ำเหลืองชนิดนอนฮอดจ์กิน","มะเร็งเม็ดเลือดขาวชนิดเฉียบพลัน","มะเร็งกระดูก","มะเร็งสมองและระบบประสาทกลาง","มะเร็งไต",
"มะเร็งมดลูก","มะเร็งถุงน้ำดี","มะเร็งกระดูกสันหลัง","มะเร็งลูกตา","มะเร็งเยื่อบุช่องคลอด","มะเร็งเยื่อบุอวัยวะเพศชาย","มะเร็งเยื่อบุอวัยวะเพศหญิง","มะเร็งต่อมหมวกไต","มะเร็งต่อมน้ำลาย","มะเร็งถุงน้ำรังไข่"
]#อันนี้ prom ของ gimini "จินี ช่วยสร้างlist python โดยในlistนั้นมีการระบุโรคมะเร็งชนิดต่างๆให้หน่อยขอเป็นเเบบภาษาทางการเเละก็ภาษาพูด ขอเเบบไม่ซ้ำด้วย"

In [ ]:
new_colcan = []
for i in range(len(comment)):#len(comment)
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(str(text)))
  Token.append('end')
  #หาโรค
  #--------------------------------------------------------
  list_cancer = []
  for i in range(len(Token)):
    if (Token[i] == "มะเร็ง")|(Token[i] == "โรคมะเร็ง"):
      list_cancer.append(Token[i]+Token[i+1])
    # else :
    #   list_cancer.append('ไม่สามารถระบุได้')
  unique_list = list(OrderedDict.fromkeys(list_cancer))
  #----------------------------------------------------------
  list_define_cancer = []
  new_list=[]
  for i in range(len(unique_list)):
    for j in range(len(name_cancar)):
      if unique_list[i]==name_cancar[j]:
        list_define_cancer.append(unique_list[i])
  #-----------------------------------------------------
  cancer_list_de2 =[]
  if len(list_define_cancer) > 0:
    if len(list_define_cancer) == 2:
      cancer_list_de2.append('เล่ามากกว่า 2 โรค')
    elif len(list_define_cancer)==1:
      cancer_list_de2.append(list_define_cancer[0])
  elif list_define_cancer==[]:
    cancer_list_de2.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
  if len(cancer_list_de2)> 0 :
    new_colcan.append(cancer_list_de2[0])
  elif len(cancer_list_de2)== 0 :
    new_colcan.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
print(new_colcan)

['ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'มะเร็งเต้านม', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'มะเร็งปอด', 'มะเร็งเต้านม', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'มะเร็งต่อมลูกหมาก', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'มะเร็งต่อมน้ำเหลือง', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'มะเร็งเต้านม', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'มะเร็งตับ', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'มะเร็งปอด', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น', 'ไม่สามารถระบุได้

In [ ]:
Genden = {'ชาย':['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ',
                 'ปู่ทวด','ตาทวด','ลุง','อาหนู','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย',
                 'พ่อตา','พ่อผม','พ่อหนู','พ่อพม','ชาย','หนุ่ม','ลช.','ผ่อ','ชย.','น้าชาย','ผ่อตา','หน.']
          ,'หญิง':['แม่','เเม่','คุณแม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า',
                   'คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า','น้า','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้',
                   'ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้','เเม่ผม','เเม่หนู','เเม่พม','แม่ผม','แม่หนู','แม่พม','สาว','หญิง','ก้อน','คลำ']}

In [ ]:
len(Genden['ชาย'])

36

In [ ]:
new_colgenden=[]
for i in range(len(comment)):
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(str(text)))
  Token.append('end')
  list_genden = []
  for j in range(len(Genden['หญิง'])):
    for k in range(len(Token)):
      if (Token[k] == Genden['ชาย'][j]):
        list_genden.append('เพศชาย')
      elif(Token[k] == Genden['หญิง'][j]):
        list_genden.append('เพศหญิง')
  genden_list =[]
  genden_list = list(OrderedDict.fromkeys(list_genden)) # ลบคำซ้ำ
  #-------------------------------------------------------------------
  list_define_genden = []
  if len(genden_list) > 0 :
    if len(genden_list) == 2:
      list_define_genden.append('เล่าทั้งสองเพศ')
    elif len(genden_list)==1:
      list_define_genden.append(genden_list[0])
  elif len(genden_list)==0:
    list_define_genden.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
  genden_list_de =[]
  genden_list_de = list(OrderedDict.fromkeys(list_define_genden))
  new_colgenden.append(genden_list_de[0])
print(new_colgenden)

In [ ]:
comment

In [ ]:
point =[]
k =[]
r=comment
for i in range(len(comment)):
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(str(text)))
  Token.append('end')
  count_text = len(Token)-1
  if new_colcan[i] =='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น':
    x=0
  elif new_colcan[i] == 'เล่ามากกว่า 2 โรค':
    x=1
  else :
    x=2
  point.append(count_text+x)
  pop = point[i]/100
  if pop >= 0.75:
    k.append('มีประโยชน์มาก')
  elif pop < 0.75 and pop >=0.5:
    k.append('มีประโยชน์')
  elif pop < 0.5 and pop >=0.25:
    k.append('อาจมีประโยชน์')
  elif pop < 0.25 :
    k.append('ไม่มีประโยชน์')
# elif max(point) > 100:
#   for i in range(len(comment)):
#     pop = point[i]/max(point)
#     if pop >= 0.75:
#       k.append('มีประโยชน์มาก')
#     elif pop < 0.75 and pop >=0.5:
#       k.append('มีประโยชน์')
#     elif pop < 0.5 and pop >=0.25:
#       k.append('อาจมีประโยชน์')
#     elif pop < 0.25 :
#       k.append('ไม่มีประโยชน์')
r['โรค'] = new_colcan
r['คะเเนน'] = point
r['ผลลัพธ์'] = k
print(r)

In [ ]:
def detect_person(comment):
    # คำที่ใช้ตรวจสอบว่ามีใครเป็นคนอยู่ในความคิดเห็น
    other = ['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ',
              'ปู่ทวด','ตาทวด','ลุง','อาหนู','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย',
              'พ่อตา','พ่อผม','พ่อหนู','พ่อพม','ชาย','หนุ่ม''แม่','เเม่','คุณแม่','มารดา','พี่สาว','น้องสาว',
              'ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า',
              'น้า','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้','เเม่ผม','เเม่หนู','เเม่พม','แม่ผม','แม่หนู','แม่พม','สาว','หญิง']
    myself = ['ผมเป็น','ผมเอง','กระผม','พมเป็น','พมเอง','กระพม','หนูเอง','หนู','ดิฉัน','ตัวเอง','ก้อน','คลำ']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in  other:
        if keyword in comment and "อาการ" not in comment :
              return 'เล่าประสบการณ์คนอื่น'
    for keyword in  myself:
        if keyword in comment:
            return 'เล่าประสบการณ์ตัวเอง'
    # หากไม่พบคำที่บ่งบอกถึงคน
    return 'ไม่ได้เล่าประสบการณ์'

In [ ]:
def detect_gender_other(comment):
    # คำที่บ่งบอกถึงเพศชาย
    male_keywords = ['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ',
                     'ปู่ทวด','ตาทวด','ลุง','อาหนู','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย',
                     'พ่อตา','พ่อผม','พ่อหนู','พ่อพม','ชาย','หนุ่ม']
    # คำที่บ่งบอกถึงเพศหญิง
    female_keywords = ['แม่','เเม่','คุณแม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า',
                       'น้า','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้','เเม่ผม','เเม่หนู','เเม่พม','แม่ผม','แม่หนู','แม่พม','สาว','หญิง']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in male_keywords:
        if keyword in comment and "อาการ" not in comment :
              return "เพศชาย"
    for keyword in female_keywords:
        if keyword in comment:
            return "เพศหญิง"
    # หากไม่พบคำที่บ่งบอกถึงเพศ
    return "ไม่ระบุเพศ"

In [ ]:
def detect_gender_self(comment):
    # คำที่บ่งบอกถึงเพศชาย
    male_keywords = ['ผมเป็น','ครับ','ผมเอง','คับ','กระผม','พมเป็น','พมเอง','กระพม']
    # คำที่บ่งบอกถึงเพศหญิง
    female_keywords = ['ค่ะ','คะ','หนูเอง','หนูเป็น','ดิฉัน','ก้อน','คลำ']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in male_keywords:
        if keyword in comment:
            return "เพศชาย"
    for keyword in female_keywords:
        if keyword in comment:
            return "เพศหญิง"
    # หากไม่พบคำที่บ่งบอกถึงเพศ
    return "ไม่ระบุเพศ"

In [ ]:
k1=[]
k2=[]
for i in comment['คำพูดโรค']:
  k1.append(detect_person(str(i)))
  if detect_person(str(i)) == 'เล่าประสบการณ์คนอื่น':
    k2.append(detect_gender_other(str(i)))
  elif detect_person(str(i)) == 'เล่าประสบการณ์ตัวเอง':
    k2.append(detect_gender_self(str(i)))
  elif detect_person(str(i)) == 'ไม่ได้เล่าประสบการณ์':
    k2.append(detect_gender_self(str(i)))
r['ใครเล่า'] = k1
r['เพศเเบ่งโดยใช้_python'] = k2

In [ ]:
r['เพศเเบ่งโดยใช้_nlp'] = new_colgenden
r['โรค'] = new_colcan

In [ ]:
r

In [ ]:
comment = r[r['คำพูดโรค']!=0]

#sentence_transformers and clustering เเบ่งว่าอันไหนมี ประโยชน์กับไม่มีประโยชน์

In [ ]:
from sentence_transformers import SentenceTransformer,util
sentences = list(comment['คำพูดโรค'])
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(sentences)
#Normalize the embeddings to unit length
Normalize_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [ ]:
clustering_model = KMeans(n_clusters=2)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)
print(len(cluster_assignment))

In [ ]:
clusternd_sentences= {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
  if cluster_id not in clusternd_sentences:
    clusternd_sentences[cluster_id] = []

  clusternd_sentences[cluster_id].append(sentences[sentence_id])

In [ ]:
len(clusternd_sentences)

In [ ]:
clusternd_sentences[1][0]

In [ ]:
len_1=[]
len_2=[]
for i in range(len(clusternd_sentences)):
    x=len(clusternd_sentences[1][i])
    len_1.append(x)
    y=len(clusternd_sentences[0][i])
    len_2.append(y)
if max(len_1) < max(len_2) :
  Pop=comment[comment['คำพูดโรค'].isin(clusternd_sentences[1])]
  Pop['โรค_clusternd'] = 'ไม่มีประโยชน์_หรือ_ให้ข้อมูลน้อยเกินไป'
  pop_use=comment[comment['คำพูดโรค'].isin(clusternd_sentences[0])]
  useful = clusternd_sentences[0]
else :
  Pop=comment[comment['คำพูดโรค'].isin(clusternd_sentences[0])]
  pop_use=comment[comment['คำพูดโรค'].isin(clusternd_sentences[1])]
  Pop['โรค_clusternd'] = 'ไม่มีประโยชน์_หรือ_ให้ข้อมูลน้อยเกินไป'
  useful = clusternd_sentences[1]

In [ ]:
Pop

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
useful_embeddings = model.encode(useful)
#Normalize the embeddings to unit length
Normalize_useful_embeddings = useful_embeddings / np.linalg.norm(useful_embeddings, axis=1, keepdims=True)
clustering_useful_model = KMeans(n_clusters=6)
clustering_useful_model.fit(Normalize_useful_embeddings)
cluster_assignment_useful = clustering_useful_model.labels_

print(cluster_assignment_useful)
print(len(cluster_assignment_useful))

In [ ]:
k_level = []
for k in range(2, len(set(pop_use['โรค']))):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(Normalize_useful_embeddings)
    labels = kmeans.labels_

    # คำนวณ Silhouette Coefficient
    silhouette_avg = silhouette_score(Normalize_useful_embeddings, labels)
    k_level.append(silhouette_avg)
print(max(k_level))

In [ ]:
import statistics
max(k_level)

In [ ]:
k_level

In [ ]:
clusternd_useful_sentences= {}
for sentence_ID, cluster_ID in enumerate(cluster_assignment_useful):
  if cluster_ID not in clusternd_useful_sentences:
    clusternd_useful_sentences[cluster_ID] = []

  clusternd_useful_sentences[cluster_ID].append(useful[sentence_ID])

In [ ]:
comment[comment['คำพูดโรค'].isin(clusternd_useful_sentences[5])]

In [ ]:
def define_Cencer_with_clusternd(use_clusternd_sentences,n_clusters):
  data = pd.DataFrame()
  for i in range(n_clusters):
    point = []
    define_C =[]
    test = comment[comment['คำพูดโรค'].isin(use_clusternd_sentences[i])]
    test_cer = list(test['โรค'])
    for j in range(len(test_cer)):
      if test_cer[j] in name_cancar:
        point.append(test_cer[j])
    if len(set(point)) == 0 :
      for x in range(len(test_cer)):
        define_C.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
    elif len(set(point)) == 1 :
      for x in range(len(test_cer)):
        define_C.append(point[0])
    elif len(set(point)) > 1:
      for x in range(len(test_cer)):
        define_C.append('มีโอกาสเป็นโรคมะเร็งมากกว่า 2 เเบบ')
    test['โรค_clusternd'] = define_C
    data = pd.concat([data,test])
  return data

In [ ]:
define_Cencer_a=define_Cencer_with_clusternd(clusternd_useful_sentences,len(name_cancar))

In [ ]:
define_Cencer_a

In [ ]:
usedata=pd.concat([Pop,define_Cencer_a])

In [ ]:
usedata

In [ ]:
data = ['A', 'B', 'C', 'A', 'B']

# แปลง list เป็น DataFrame
df = pd.DataFrame(data, columns=['data'])

# One-hot encoding
dummy_variables = pd.get_dummies(df['data'])

# แสดงผล

In [ ]:
dummy_variables

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# ตัวอย่าง list
data = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]]

# แปลง list เป็น DataFrame
df = pd.DataFrame(data, columns=['x', 'y'])

# ลองใช้ค่า k ที่แตกต่างกัน
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(df[['x', 'y']])
    labels = kmeans.labels_

    # คำนวณ Silhouette Coefficient
    silhouette_avg = silhouette_score(df[['x', 'y']], labels)
    print(f"k = {k}, Silhouette Coefficient = {silhouette_avg}")